# Codificación de las variables categoricas, escalado y modelo

In [1]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
# conda install category_encoders
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix

from sklearn import metrics

Leemos los datos de entreno y test construidor en el anterior Notebook.

In [2]:
pd_fraud_train=pd.read_csv("../data/train_pd_data_preprocessing_missing_outlier.csv")

In [3]:
pd_fraud_train

,Unnamed: 0,income,device_fraud_count,device_distinct_emails_8w,keep_alive_session,device_os,session_length_in_minutes,source,foreign_request,proposed_credit_limit,...,zip_count_4w,payment_type,intended_balcon_amount,days_since_request,customer_age,current_address_months_count,prev_address_months_count,name_email_similarity,month,fraud_bool
0,104793,0.1,0,1,0,windows,2.126967,INTERNET,0,500.0,...,1485,AB,-0.925736,0.021851,40,2,8,0.898839,3,0
1,131379,0.1,0,1,1,linux,14.136746,INTERNET,0,200.0,...,1092,AC,-1.412097,0.001366,30,8,88,0.547038,3,0
2,440665,0.5,0,1,1,linux,8.533374,INTERNET,0,200.0,...,979,AB,-1.220475,0.004143,30,17,-1,0.104121,1,0
3,217656,0.8,0,1,1,linux,4.582946,INTERNET,0,990.0,...,1667,AC,-0.251319,0.000040,20,24,-1,0.554209,3,0
4,401950,0.2,0,1,1,macintosh,3.942180,INTERNET,0,200.0,...,3889,AB,-0.455009,0.006409,10,17,32,0.681949,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,859711,0.8,0,2,1,other,1.730376,INTERNET,0,200.0,...,821,AB,-1.116012,0.025359,20,46,-1,0.276809,5,0
799996,897142,0.1,0,1,0,linux,8.594472,INTERNET,0,200.0,...,1178,AB,-0.450084,0.024832,40,29,-1,0.403988,4,0
799997,35326,0.8,0,1,0,other,0.922873,INTERNET,0,200.0,...,958,AD,-0.735873,0.007345,30,30,-1,0.562414,7,0
799998,845054,0.3,0,1,1,other,2.521377,INTERNET,0,200.0,...,340,AD,49.302081,0.003609,30,8,29,0.499213,5,0


In [4]:
pd_fraud_test=pd.read_csv("../data/test_pd_data_preprocessing_missing_outlier.csv")

In [5]:
pd_fraud_test

,Unnamed: 0,income,device_fraud_count,device_distinct_emails_8w,keep_alive_session,device_os,session_length_in_minutes,source,foreign_request,proposed_credit_limit,...,zip_count_4w,payment_type,intended_balcon_amount,days_since_request,customer_age,current_address_months_count,prev_address_months_count,name_email_similarity,month,fraud_bool
0,267199,0.1,0,1,0,linux,5.023156,INTERNET,0,200.0,...,2555,AB,-0.507390,0.029926,20,8,12,0.306713,2,0
1,863338,0.6,0,1,1,linux,1.675085,INTERNET,0,200.0,...,2624,AC,-1.218131,0.002462,30,42,-1,0.456211,5,0
2,445129,0.9,0,1,0,linux,1.338952,INTERNET,0,200.0,...,885,AB,-1.536734,0.029170,50,7,65,0.015233,1,0
3,949635,0.2,0,1,1,windows,8.170057,INTERNET,0,200.0,...,1700,AC,-1.727492,0.013426,50,93,-1,0.932129,4,0
4,271851,0.7,0,1,0,windows,17.164353,INTERNET,0,500.0,...,2771,AB,-0.480861,0.004021,20,28,-1,0.253185,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,169659,0.4,0,1,0,windows,1.441572,INTERNET,0,200.0,...,1442,AA,26.207088,33.920958,20,121,-1,0.441344,3,0
199996,199272,0.7,0,1,1,other,2.911984,INTERNET,0,1500.0,...,4041,AA,34.329905,0.037431,30,145,-1,0.159174,3,0
199997,71026,0.4,0,1,1,linux,13.706277,INTERNET,0,200.0,...,633,AB,-1.108422,0.000180,20,1,311,0.544298,7,0
199998,589697,0.7,0,1,0,windows,4.950243,INTERNET,0,1500.0,...,902,AB,-1.568353,0.018825,40,34,-1,0.297059,6,0


Eliminamos la variable "Unnamed: 0" que se creo al guardar el dataset en el anterior notebook y leerlo en el actual.

In [6]:
pd_fraud_train =pd_fraud_train.drop(columns=["Unnamed: 0"])

In [7]:
pd_fraud_test=pd_fraud_test.drop(columns=["Unnamed: 0"])

Listamos las variables que son categóricas y las numéricas. Identificamos el target.

In [8]:
lista_variables_categoricas = ["payment_type","employment_status","housing_status","email_is_free","phone_home_valid",
                                "phone_mobile_valid","has_other_cards","foreign_request","source","device_os","keep_alive_session", "fraud_bool"]

lista_variables_numericas = ["income", "name_email_similarity", "prev_address_months_count", "current_address_months_count", "customer_age", "days_since_request",
                            "intended_balcon_amount", "zip_count_4w", "velocity_6h", "velocity_24h", "velocity_4w", "bank_branch_count_8w", "date_of_birth_distinct_emails_4w",
                            "credit_risk_score", "bank_months_count", "proposed_credit_limit", "session_length_in_minutes", "device_distinct_emails", "device_fraud_count",
                            "month"]
target = 'fraud_bool'

Asignamos en pd_fraud_train y a pd_fraud_test las variables categóricas en función del enunciado. Tambien ponemos la 
variable objetivo en String.

In [9]:
for columna in lista_variables_categoricas:
    pd_fraud_train[columna] = pd_fraud_train[columna].astype('category')

In [10]:
for columna in lista_variables_categoricas:
    pd_fraud_test[columna] = pd_fraud_test[columna].astype('category')

In [11]:
pd_fraud_train[target] = pd_fraud_train[target].astype(str)

In [12]:
pd_fraud_test[target] = pd_fraud_test[target].astype(str)

In [13]:
pd_fraud_train.columns

Index(['income', 'device_fraud_count', 'device_distinct_emails_8w',
       'keep_alive_session', 'device_os', 'session_length_in_minutes',
       'source', 'foreign_request', 'proposed_credit_limit', 'has_other_cards',
       'bank_months_count', 'phone_mobile_valid', 'phone_home_valid',
       'housing_status', 'email_is_free', 'credit_risk_score',
       'employment_status', 'date_of_birth_distinct_emails_4w',
       'bank_branch_count_8w', 'velocity_4w', 'velocity_24h', 'velocity_6h',
       'zip_count_4w', 'payment_type', 'intended_balcon_amount',
       'days_since_request', 'customer_age', 'current_address_months_count',
       'prev_address_months_count', 'name_email_similarity', 'month',
       'fraud_bool'],
      dtype='object')

In [14]:
pd_fraud_test

,income,device_fraud_count,device_distinct_emails_8w,keep_alive_session,device_os,session_length_in_minutes,source,foreign_request,proposed_credit_limit,has_other_cards,...,zip_count_4w,payment_type,intended_balcon_amount,days_since_request,customer_age,current_address_months_count,prev_address_months_count,name_email_similarity,month,fraud_bool
0,0.1,0,1,0,linux,5.023156,INTERNET,0,200.0,0,...,2555,AB,-0.507390,0.029926,20,8,12,0.306713,2,0
1,0.6,0,1,1,linux,1.675085,INTERNET,0,200.0,1,...,2624,AC,-1.218131,0.002462,30,42,-1,0.456211,5,0
2,0.9,0,1,0,linux,1.338952,INTERNET,0,200.0,0,...,885,AB,-1.536734,0.029170,50,7,65,0.015233,1,0
3,0.2,0,1,1,windows,8.170057,INTERNET,0,200.0,0,...,1700,AC,-1.727492,0.013426,50,93,-1,0.932129,4,0
4,0.7,0,1,0,windows,17.164353,INTERNET,0,500.0,0,...,2771,AB,-0.480861,0.004021,20,28,-1,0.253185,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.4,0,1,0,windows,1.441572,INTERNET,0,200.0,1,...,1442,AA,26.207088,33.920958,20,121,-1,0.441344,3,0
199996,0.7,0,1,1,other,2.911984,INTERNET,0,1500.0,0,...,4041,AA,34.329905,0.037431,30,145,-1,0.159174,3,0
199997,0.4,0,1,1,linux,13.706277,INTERNET,0,200.0,0,...,633,AB,-1.108422,0.000180,20,1,311,0.544298,7,0
199998,0.7,0,1,0,windows,4.950243,INTERNET,0,1500.0,0,...,902,AB,-1.568353,0.018825,40,34,-1,0.297059,6,0


Comprobamos que los tipos estan tal y como hemos planteado anteriormente.

In [15]:
pd_fraud_train.dtypes

income                               float64
device_fraud_count                     int64
device_distinct_emails_8w              int64
keep_alive_session                  category
device_os                           category
session_length_in_minutes            float64
source                              category
foreign_request                     category
proposed_credit_limit                float64
has_other_cards                     category
bank_months_count                      int64
phone_mobile_valid                  category
phone_home_valid                    category
housing_status                      category
email_is_free                       category
credit_risk_score                      int64
employment_status                   category
date_of_birth_distinct_emails_4w       int64
bank_branch_count_8w                   int64
velocity_4w                          float64
velocity_24h                         float64
velocity_6h                          float64
zip_count_

Hacemos un recuento de cuantas operaciones en el total del dataset han sido fraudulentas y cuantas no. Lo hacemos tanto con 
los valores 0 y 1 como con los valores Fraude y No fraude para que sea más legible.

In [16]:
pd_fraud_train['fraud_bool'].value_counts()

fraud_bool
0    791177
1      8823
Name: count, dtype: int64

In [17]:
# Creamos una copia del DataFrame original para evitar modificarlo directamente
pd_fraud_train_str = pd_fraud_train.copy()

# Reemplazamoos los valores en la columna 'fraud_bool'
pd_fraud_train_str['fraud_bool'] = pd_fraud_train_str['fraud_bool'].map({"0": "No fraude", "1": "Fraude"})

pd_fraud_test_str = pd_fraud_test.copy()

pd_fraud_test_str['fraud_bool'] = pd_fraud_test_str['fraud_bool'].map({"0": "No fraude", "1": "Fraude"})



In [18]:
pd_fraud_train_str['fraud_bool'].value_counts()

fraud_bool
No fraude    791177
Fraude         8823
Name: count, dtype: int64

In [19]:
pd_fraud_test_str['fraud_bool'].value_counts()

fraud_bool
No fraude    197794
Fraude         2206
Name: count, dtype: int64

In [20]:
pd_fraud_train['fraud_bool'].value_counts()

fraud_bool
0    791177
1      8823
Name: count, dtype: int64

Dividimos nuevamente pd_fraud train y pd_fraud test en conjuntos de entrenamiento y prueba. Utilizaremos X_train e y_train para 
entrenar el modelo mientras que evaluaremos la capacidad del mismo con X_test e y_test. La razón de esta división es para 
evitar el overfitting. Al mantener un conjunto de prueba separado que el modelo no ve durante el entrenamiento,
podemos tener una mejor evaluación de su rendimiento en la práctica.

In [21]:
X_train = pd_fraud_train.drop('fraud_bool',axis=1)
X_test = pd_fraud_test.drop('fraud_bool',axis=1)
y_train = pd_fraud_train['fraud_bool']
y_test = pd_fraud_test['fraud_bool']

In [22]:
X_train.dtypes

income                               float64
device_fraud_count                     int64
device_distinct_emails_8w              int64
keep_alive_session                  category
device_os                           category
session_length_in_minutes            float64
source                              category
foreign_request                     category
proposed_credit_limit                float64
has_other_cards                     category
bank_months_count                      int64
phone_mobile_valid                  category
phone_home_valid                    category
housing_status                      category
email_is_free                       category
credit_risk_score                      int64
employment_status                   category
date_of_birth_distinct_emails_4w       int64
bank_branch_count_8w                   int64
velocity_4w                          float64
velocity_24h                         float64
velocity_6h                          float64
zip_count_

Comprobamos el numero de columnas en X_train ya que posteriormente al hacer el OneHotEnconder queremos saber cuanto han
aumentado el número de columnas.

In [23]:
len(list(X_train.columns))

31

Hacemos una lista con todos los nombres de las columnas que contienen variables "object" o "category" y otra lista que contengan
todas aquellas columnas que no esten en la primera lista.

In [24]:

list_columns_cat = list(X_train.select_dtypes(include=['category', 'object']).columns)
print(list_columns_cat)


['keep_alive_session', 'device_os', 'source', 'foreign_request', 'has_other_cards', 'phone_mobile_valid', 'phone_home_valid', 'housing_status', 'email_is_free', 'employment_status', 'payment_type']


In [25]:

list_other = list(set(X_train.columns)-set(list_columns_cat))

In [26]:
list_columns_cat

['keep_alive_session',
 'device_os',
 'source',
 'foreign_request',
 'has_other_cards',
 'phone_mobile_valid',
 'phone_home_valid',
 'housing_status',
 'email_is_free',
 'employment_status',
 'payment_type']

Creamos un objeto OneHotEncoder de la biblioteca category_encoders.Solo aplicamos la transformacion a las columnas categóricas.
Con esto lo que hacemos es convertir las distintas categorias de una columna en columnas. Cada columna tendrá un '1' si la categoría original era esa y '0' 
de lo contrario.

Hemos elegido OneHotEnconder ya que nuestras variables categóricas no tienen
excesivas categorías.

Posterimente entrenamos con los datos X_train e y_train.Ahora model 
contiene el aprendzaje conseguido a través de los datos de entrenamiento.

In [27]:
ohe = ce.OneHotEncoder(cols=list_columns_cat)
model = ohe.fit(X_train, y_train)

In [28]:
model

OneHotEncoder(cols=['keep_alive_session', 'device_os', 'source',
                    'foreign_request', 'has_other_cards', 'phone_mobile_valid',
                    'phone_home_valid', 'housing_status', 'email_is_free',
                    'employment_status', 'payment_type'])

Utilizamos el modelo OneHotEncoder que se ajustó previamente a los datos de entrenamiento para transformar tanto el conjunto de
entrenamiento (X_train e y_trian) como el conjunto de prueba (X_test e y_test).

In [29]:
X_train_t = model.transform(X_train,y_train)
X_test_t = model.transform(X_test, y_test)
# X_train_t.columns = list_columns_cat
# X_test_t.columns = list_columns_cat

Ahora tenemos conjuntos de datos de entrenamiento y prueba (X_train_t y X_test_t) que están listos para ser utilizados en 
un modelo de aprendizaje automático, con todas las variables categóricas codificadas adecuadamente. 

Es muy importante que en ambos conjuntos se utilice la misma transformación ya que esto nos permite mantener 
la consistencia de los datos al entrenar y evaluar el modelo.

Calculamos el número de columnas en el DataFrame X_train_t, así sabemos el nuevo número de columnas que se han creado.

In [30]:
len(list(X_train_t.columns))

58

In [31]:
X_train_t.dtypes.to_dict()

{'income': dtype('float64'),
 'device_fraud_count': dtype('int64'),
 'device_distinct_emails_8w': dtype('int64'),
 'keep_alive_session_1': dtype('int64'),
 'keep_alive_session_2': dtype('int64'),
 'device_os_1': dtype('int64'),
 'device_os_2': dtype('int64'),
 'device_os_3': dtype('int64'),
 'device_os_4': dtype('int64'),
 'device_os_5': dtype('int64'),
 'session_length_in_minutes': dtype('float64'),
 'source_1': dtype('int64'),
 'source_2': dtype('int64'),
 'foreign_request_1': dtype('int64'),
 'foreign_request_2': dtype('int64'),
 'proposed_credit_limit': dtype('float64'),
 'has_other_cards_1': dtype('int64'),
 'has_other_cards_2': dtype('int64'),
 'bank_months_count': dtype('int64'),
 'phone_mobile_valid_1': dtype('int64'),
 'phone_mobile_valid_2': dtype('int64'),
 'phone_home_valid_1': dtype('int64'),
 'phone_home_valid_2': dtype('int64'),
 'housing_status_1': dtype('int64'),
 'housing_status_2': dtype('int64'),
 'housing_status_3': dtype('int64'),
 'housing_status_4': dtype('int64

# Conclusión

Para concluir, con este preprocesamiento hemos establecido en que partes del dataset debemos poner el foco.
Asignamos a cada variable su tipo de dato, identificamos cuales de ellas son las que tienen una mayor relación con el fraude(a través de gráficos, correlaciones, cramer´s_v...) y hemos identificado los valores nulos y 
outliers para su posterior tratamiento. Además también hemos separado los datos en train 
y test y codificado las variables categóricas.

Ahora nuestros datos están listos para ser utilizados en un modelo de aprendizaje automático, con todas las variables
categóricas codificadas adecuadamente.